# Usando a Biblioteca Ray rllib

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/capExtra/capX-ray-rllib.ipynb)


In [ ]:
!pip install "ray[rllib]"

In [ ]:
from IPython.display import clear_output
import tensorboard

%load_ext tensorboard

from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print

## 1 - Tensorboard

Acompanhe, em especial, a métrica `ray/tune/episode_reward_mean` ou `ray/tune/sampler_results
/episode_reward_mean`.

In [ ]:
%tensorboard --logdir=~/ray_results  # no Colab, os resultados ficam em "root/ray_results"

## 2 - Treinamento com o PPO

In [ ]:
ENV_NAME = 'CartPole-v1'  # 'MountainCarContinuous-v0'

In [ ]:
algo = (
    PPOConfig()
    .rollouts(num_rollout_workers=0)
    .resources(num_gpus=1)           # atenção: 1 GPU
    .environment(env=ENV_NAME)
    .build()
)

# treina 4 mil passos por iteração
for i in range(20):
    result = algo.train()
    clear_output()

    # result é um dicionário com muitas informações do treinamento
    # se quiser tamanho e recompensa de cada episódio, acesse result['sampler_results']['hist_stats'].keys()
    print(pretty_print(result))

    if (i+1) % 5 == 0:
        checkpoint_dir = algo.save()
        print(f"Checkpoint saved in directory {checkpoint_dir}")

## 3 - Execução do Agente Treinado

In [ ]:
import gymnasium as gym

env = gym.make(ENV_NAME)

for i in range(5):
  episode_reward = 0
  terminated = truncated = False
  obs, info = env.reset()

  while not terminated and not truncated:
      action = algo.compute_single_action(obs)
      obs, reward, terminated, truncated, info = env.step(action)
      episode_reward += reward

  print(f'Episode {i+1} -> {episode_reward=}')
